In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.cluster import KMeans
from sklearn.ensemble import BaggingRegressor

import plotly.express as px

import plotly.graph_objects as go

from scipy.linalg import toeplitz, block_diag

from spe.mse_estimator import ErrorComparer, gen_rbf_X, create_clus_split
from spe.relaxed_lasso import RelaxedLasso, BaggedRelaxedLasso

In [2]:
err_cmp = ErrorComparer()

In [99]:
niter = 200

n=10**2
p=200
s=30

k=10
alpha=1.
lambd=.45

In [100]:
nx = ny = int(np.sqrt(n))
xs = np.linspace(0, 10, nx)
ys = np.linspace(0, 10, ny)
c_x, c_y = np.meshgrid(xs, ys)
c_x = c_x.flatten()
c_y = c_y.flatten()

In [101]:
rho = 0.5

D = np.abs(c_x - c_x[:,None]) + np.abs(c_y - c_y[:,None])
Sigma_t = rho**D

# m = 20
# c = np.zeros(n)
# c[:m] = [rho**i for i in np.arange(m)]
# Sigma_t = toeplitz(c)

# b = n / m
# c = [rho**i for i in np.arange(m)] 
# Sigma_t = block_diag(*[toeplitz(c) for _ in np.arange(b)])

Chol_t = np.linalg.cholesky(Sigma_t)
# Sigma_t = np.eye(n)
# Chol_t = np.eye(n)

In [102]:
# c_x = np.random.uniform(0,10,size=n)
# c_y = np.random.uniform(0,10,size=n)

# rn = int(np.sqrt(n))
# ctr_x = np.random.uniform(0,10,size=rn)
# ctr_y = np.random.uniform(0,10,size=rn)
# c_x = np.concatenate([c + .35 * np.random.randn(rn) for c in ctr_x])
# c_y = np.concatenate([c + .35 * np.random.randn(rn) for c in ctr_y])

X = gen_rbf_X(c_x, c_y, p)
idx = create_clus_split(int(np.sqrt(n)), int(np.sqrt(n)))
X_sample = X[idx,:]
Chol_t_sample = Chol_t[idx,:][:,idx]
c_x_sample = c_x[idx]
c_y_sample = c_y[idx]

beta = np.zeros(p)
idx = np.random.choice(p,size=s)
beta[idx] = np.random.uniform(-1,1,size=s)

In [103]:
# xv, yv = np.meshgrid(np.arange(nx), np.arange(ny))
# pts = np.stack([xv.ravel(), yv.ravel()]).T
# n = nx*ny
# rn = int(np.sqrt(n))
# ctr = np.random.choice(pts.shape[0], size=rn, replace=True)
# ctr = pts[ctr]
# tr_idx = np.vstack([[pt + np.array((1.25*np.random.randn(2)).astype(int)) for _ in np.arange(rn)] for pt in ctr])
# tr_idx = np.maximum(0, tr_idx)
# tr_idx[:,0] = cx = np.minimum(nx-1, tr_idx[:,0])
# tr_idx[:,1] = cy = np.minimum(ny-1, tr_idx[:,1])
# tr_idx = np.unique(np.ravel_multi_index(tr_idx.T, (nx,ny)))
# tr_idx.shape

In [104]:
(test_err,
 kfcv_err,
 spcv_err,
 rela_err) = err_cmp.compareRelaxedLassoTrTsFair(n=n,
                                           p=p,
                                           X=X_sample,
                                           beta=beta,
                                           coord=np.stack([c_x_sample, c_y_sample]).T,
                                           Chol_t=Chol_t_sample,
                                           niter=niter,
                                           lambd=lambd,
                                           alpha=alpha,
                                           k=k)
    

0
0.7941176470588235
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190


In [105]:
#%debug

> /Users/kevinfry/opt/anaconda3/envs/blur/lib/python3.8/site-packages/numpy/core/fromnumeric.py(2450)all()
   2448 
   2449     """
-> 2450     return _wrapreduction(a, np.logical_and, 'all', axis, None, out,
   2451                           keepdims=keepdims, where=where)
   2452 

ipdb> quit


In [106]:
# groups = KMeans(n_clusters=10).fit(tr_idx).labels_

In [107]:
# px.scatter(pd.DataFrame({'x': tr_idx[:,0], 'y': tr_idx[:,1], 'g': groups}), x='x', y='y', color='g')

In [108]:
risk = test_err.mean()
risk_kfcv = kfcv_err.mean()
risk_spcv = spcv_err.mean()
# risk_rela = rela_err.mean()
risk_brel = rela_err.mean()
risk, risk_kfcv, risk_spcv, risk_brel

(0.2698667599727253,
 0.22533424914965886,
 0.253181538639694,
 0.27801303629770335)

In [109]:
df = pd.DataFrame({'GenCp': (rela_err.T),# - test_err.T) / test_err.T,
                   'KFCV': (kfcv_err.T),# - test_err.T) / test_err.T,
                   'SPCV': (spcv_err.T)})# - test_err.T) / test_err.T})

In [114]:
(pd.DataFrame({'GenCp': (rela_err.T),
               'KFCV': (kfcv_err.T),
               'SPCV': (spcv_err.T)})/risk).to_csv('~/Downloads/rlf_df.csv')

In [110]:
fig_rela = px.box((df - risk) / risk, 
             labels={
                     "variable": "Method",
                     "value": "Relative MSE"
                     },
             title="lin correction",
             points=False)
fig_rela.update_traces(boxmean=True)

fig_rela.add_hline(y=0., line_color='red')

In [111]:
fig = go.Figure(data=go.Scatter(
        x=[0, 1, 2],
        y=((df - risk)/risk).mean(),
        error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=((df - risk)/risk).std(),
            visible=True)
    ))
fig.show()

In [112]:
pd.DataFrame((df/risk).mean())

,0
GenCp,1.030186
KFCV,0.834983
SPCV,0.938172


In [139]:
fig = go.Figure()
fig.add_hline(y=1., line_color='red')
fig.add_trace(go.Bar(
    name='Control',
    x=['GenCp', 'KFCV', 'SPCV'], y=(df/risk).mean(),
    marker_color=px.colors.qualitative.Plotly,
    text=np.around((df/risk).mean(),3),
    textposition='outside',
    error_y=dict(
        type='data',
        color='black',
        symmetric=False,
        array=(df/risk).quantile(.75),
        arrayminus=(df/risk).quantile(.25))
))
# fig.add_trace(go.Bar(
#     name='Experimental',
#     x=['GenCp', 'KFCV', 'SPCV'], y=(df/risk).mean(),
#     error_y=dict(type='data', array=[1, 2])
# ))
# fig.update_layout(barmode='group')
fig.update_layout(
    title="Relaxed Lasso Test Error Estimates",
    xaxis_title="Method",
    yaxis_title="Relative MSE",
#     legend_title="Legend Title",
#     font=dict(
#         family="Courier New, monospace",
#         size=18,
#         color="RebeccaPurple"
#     )
)
# ticktext= []
# for i in np.arange()
# fig.update_layout(
#     yaxis = dict(
#         tickmode = 'linear',
#         tick0 = 0.,
#         dtick = 1.,
# #         ticktext = ticktext,
#     )
# )
fig.show()